#### Configure your AI service credentials

Use [this notebook](setup.ipynb) first, to choose whether to run these notebooks with OpenAI or Azure OpenAI,
and to save your credentials in the configuration file.

In [6]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"

// Instantiate the Kernel
using Microsoft.SemanticKernel;

//Create Kernel builder
var builder = new KernelBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

IKernel kernel = builder.Build();


Restore sources https://nuget.pkg.github.com/Microsoft.SemanticKernel/index.json https://nuget.pkg.github.com/microsoft/index.json https://nuget.pkg.github.com/microsoft/Microsoft.SemanticKernel/index.json Installed Packages Microsoft.SemanticKernel, 0.24.230918.1-preview

#### Get User Input and Run a SemanticFunction

It executes call to generative model a specified input as a prompt. If we ask what time is it, the model will not be able to response to that. That is where **native functions** come into play.

In [ ]:
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
using Microsoft.SemanticKernel;

var name = await InteractiveKernel.GetInputAsync("Question: ");

// response is limited to 2000 tokens
// tell me a joke
// was the joke
// each chat session is stateless
Console.WriteLine(await kernel.InvokeSemanticFunctionAsync(name, maxTokens: 2000));
Console.WriteLine();

SK supports multiple kinds of functions that can be invoked:

1. ["semantic functions"](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/semantic-functions/inline-semantic-functions?tabs=Csharp) (text-based interactions with AI) 
2. ["native functions"](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/plugins/native-functions/using-the-skfunction-decorator?tabs=Csharp) (.NET methods that can do anything C# code can do)

#### Create a **native function** to tell the time.

In [10]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SkillDefinition;


// 1. register a native function that returns time and date
kernel.RegisterCustomFunction(
    SKFunction.FromNativeFunction(
        () => $"{DateTime.UtcNow:r}",
        "DateTime", "Now",
        "Gets the current date and time"));
        
// 2. register a semintic function that uses the native function       
ISKFunction qa = kernel.CreateSemanticFunction("""
    The current date and time is {{ datetime.now }}.
    {{ $input }}
    """, maxTokens: 2000);

// what day of the week is it?
// what time is it?
// what is the date?
var name = await InteractiveKernel.GetInputAsync("Question: ");
Console.WriteLine(await qa.InvokeAsync(name, kernel.Skills));
Console.WriteLine();

The date is October 9th, 2023.

